In [16]:
from timeit import timeit
import altair as alt
import pandas as pd

# what are the practical implications of these observations?

alt.data_transformers.disable_max_rows()

n = 100000
batch_size = 100

def make_lists(n):
    return [list(range(i)) for i in range(batch_size, n + batch_size, 50)]

funcs = {
    "pop()": lambda l: l.pop(),
    "pop(0)": lambda l: l.pop(0),
    "insert(0, x)": lambda l: l.insert(0, None),
    "append(x)": lambda l: l.append(None)
}

df = pd.DataFrame({
    k: list(map(lambda l: timeit(lambda: v(l), number=batch_size), make_lists(n)))
    for k,v in funcs.items()
})

df = df.rolling(batch_size, center=True).mean().dropna()
df = df.reset_index().melt(id_vars="index", value_vars=funcs.keys())
df = df.rename(columns={"index": "List Size", "value": "Time", "variable": "Method"})

alt.Chart(df).mark_line().encode(
    x="List Size",
    y="Time",
    color="Method"
)

alt.Chart(...)